# Software Analysis - 01 - Binary Size

In cases of constrained storage space, e.g. on embedded devices, or limited transmission bandwidth, e.g. for mobile app download, it might be necessary to optimize the size of a binary executable.

In the following, the analysis and optimization of the binary size of a compiled application is explained.

## Handling Command Line Calls in Python

For shell command calls, the Python module ``subprocess`` is used (https://docs.python.org/3/library/subprocess.html).

In [ ]:
import subprocess

In the current working directory, you will find three "Hello World!" programs in three different programming languages. They will be the first targets of this binary size analysis.

You can list the directories by calling the ``ls`` command with the ``-l`` option in ``subprocess.run()``.

In [ ]:
ls = subprocess.run(['ls', '-l'], capture_output=True, text=True)
print(ls.stdout)

## Get File Size of Binaries in Python

One way to compare the size of binaries is to look at the file size of these executables.

In Python, the function ``os.path.getsize()`` can be used to measure file size (https://docs.python.org/3/library/os.path.html).

In [ ]:
import os

Let's use it on some Linux binaries.

In [ ]:
file_path = '/usr/bin/ls'
print('{:<24}'.format(file_path), '-->', '{:>9}'.format(os.path.getsize(file_path)), 'bytes')
file_path = '/usr/bin/cat'
print('{:<24}'.format(file_path), '-->', '{:>9}'.format(os.path.getsize(file_path)), 'bytes')

## Get Sections' Sizes of Binaries

A second way to measure binary size is the analysis of sections in a binary.

While these also contain sections that are only allocated at runtime and do not hold data, like ``.bss``, this might be helpful when optimizing code and static data.

On Linux, the ``size`` command with option ``-A`` can show the size of different sections in an executable.

The following sections might be among those of interest:
- ``.text`` contains all code
- ``.rodata`` represents read-only data
- ``.data`` holds writable data

In [ ]:
size = subprocess.run(['size', '-A', '/usr/bin/ls'], cwd='./', capture_output=True, text=True)
print(size.stdout)

In order to filter the lengthy ``size -A`` output, one can use regular expressions with the Python module ``re`` (https://docs.python.org/3/library/re.html).

In [ ]:
import re

The following example shows filtering for sections ``.text``, ``.rodata``, ``.data`` and ``.bss``.

In [ ]:
size = subprocess.run(['size', '-A', '/usr/bin/ls'], cwd='./', capture_output=True, text=True)
print('\n'.join(re.findall('section.*|\.text\s+.*|\.rodata\s+.*|\.data\s+.*|\.bss\s+.*', size.stdout)))

## Analysis of ''Hello World'' Binaries in C, C++, and Rust

In the section, ''Hello World'' applications in the languages C, C++, and Rust will be compiled in different ways and analyzed for their binary sizes.

### Folder Structure and Files

The following code lists the contents of each ``hello_world_*`` directory.

In [ ]:
print('### C ###')
ls = subprocess.run(['ls', '-l'], cwd='./hello_world_c/', capture_output=True, text=True)
print(ls.stdout)

print('### C++ ###')
ls = subprocess.run(['ls', '-l'], cwd='./hello_world_cpp/', capture_output=True, text=True)
print(ls.stdout)

print('### Rust ###')
ls = subprocess.run(['ls', '-l'], cwd='./hello_world_rust/', capture_output=True, text=True)
print(ls.stdout)

The command ``cat`` can be used to print the contents of the files ``hello_world.c``, ``hello_world.cpp``, and ``src/main.rs``!

In [ ]:
print('### C ###')
cat = subprocess.run(['cat', 'hello_world.c'], cwd='./hello_world_c/', capture_output=True, text=True)
print(cat.stdout)

print('### C++ ###')
cat = subprocess.run(['cat', 'hello_world.cpp'], cwd='./hello_world_cpp/', capture_output=True, text=True)
print(cat.stdout)

print('### Rust ###')
cat = subprocess.run(['cat', 'src/main.rs'], cwd='./hello_world_rust/', capture_output=True, text=True)
print(cat.stdout)

### Compiler Options for C and C++

The size of a binary can vary significantly, depending on the used compiler options.

The following examples detail differnt compilation approaches for applications in C, C++ and Rust.

The ``Makefile`` in the C and C++ cases contains different options to compile the ``hello_world`` binary.

Have a look at the contents with the code of the following cell.

In [ ]:
print('### C - Makefile ###')
cat = subprocess.run(['cat', 'Makefile'], cwd='./hello_world_c/', capture_output=True, text=True)
print(cat.stdout)

print('### C++ - Makefile ###')
cat = subprocess.run(['cat', 'Makefile'], cwd='./hello_world_cpp/', capture_output=True, text=True)
print(cat.stdout)

In the default case, ``gcc`` compiles a dynamically linked binary from the provide C code.

The option ``-Os`` makes ``gcc`` optimize in favor of binary size.

The option ``-static`` enables the generation of statically linked binary.

Of course, both options can be combined to obtain a statically linked binary optimized for size.

Additionaly, the option ``-flto`` activates the Link-Time Optimizer (LTO).

The C++ compiler ``g++`` takes almost the same options to achieve similar optimization.

Let's compile all versions for C and C++:

In [ ]:
print('### Compile C Binaries ###')
make = subprocess.run(['make', 'all'], cwd='./hello_world_c/')
print('Return Code:', make.returncode)

In [ ]:
print('### Compile C++ Binaries ###')
make = subprocess.run(['make', 'all'], cwd='./hello_world_cpp/')
print('Return Code:', make.returncode)

### Compiler Options for Rust

For Rust, optimizations can be handled by calling ``cargo`` with different options and setting parameters in the ``Cargo.toml`` file.

By default, Rust compiles statically linked binaries.

Also, if no options are given, the ``debug`` profile is used, i.e. debug information is still contained in the binary and optimization is not set to the highest level.

In [ ]:
print('### Compile Rust Binary (debug) ###')
cargo = subprocess.run(['cargo', 'build'], cwd='./hello_world_rust/')
print('Return Code:', cargo.returncode)

Adding ``--release`` to the ``cargo`` call will set the highest optimization level.

Further, the release profile in the ``Cargo.toml`` file can be extended, e.g. to enable the Link-Time Optimizer (LTO):<br>
``[profile.release]``<br>
``lto = true``<br>
``panic = 'abort'``<br>

In [ ]:
print('### Compile Rust Binary (release) ###')
cargo = subprocess.run(['cargo', 'build', '--release'], cwd='./hello_world_rust/')
print('Return Code:', cargo.returncode)

However, if one uses the tool ``ldd`` to look at the compiled binaries, one can notice that - under Linux - Rust does not build fully statically linked binaries.

In [ ]:
ldd = subprocess.run(['ldd', 'hello_world'], cwd='./hello_world_rust/target/release/', capture_output=True, text=True)
print(ldd.stdout)

If one wants to compare Rust binaries with C and C++ binaries, one has to achieve completely statically linked binaries.

Therefore, one has to install a specific additional target:

``rustup target add x86_64-unknown-linux-musl``

Afterwards, a fully statically linked binary can be compiled as follows.

In [ ]:
print('### Compile Rust Binary (release, fully statically linked) ###')
cargo = subprocess.run(['cargo', 'build', '--target', 'x86_64-unknown-linux-musl', '--release'], cwd='./hello_world_rust/')
print('Return Code:', cargo.returncode)

In [ ]:
ldd = subprocess.run(['ldd', 'hello_world'], cwd='./hello_world_rust/target/x86_64-unknown-linux-musl/release/', capture_output=True, text=True)
print(ldd.stdout)

### Stripping Binaries

The following list contains all compiled binaries.

In [ ]:
binaries = [
    ['./hello_world_c/', 'hello_world_dyn'],
    ['./hello_world_c/', 'hello_world_dyn_sizeopt'],
    ['./hello_world_c/', 'hello_world_stat'],
    ['./hello_world_c/', 'hello_world_stat_sizeopt'],
    ['./hello_world_cpp/', 'hello_world_dyn'],
    ['./hello_world_cpp/', 'hello_world_dyn_sizeopt'],
    ['./hello_world_cpp/', 'hello_world_stat'],
    ['./hello_world_cpp/', 'hello_world_stat_sizeopt'],
    ['./hello_world_rust/target/debug/', 'hello_world'],
    ['./hello_world_rust/target/release/', 'hello_world'],
    ['./hello_world_rust/target/x86_64-unknown-linux-musl/release/', 'hello_world'],
]

Last, but not least, one can use the tool ``strip`` to clean the executables from all unnecessary information.

In [ ]:
print('### Stripping Hello World Binaries ###')
for [directory, binfile] in binaries:
  strip = subprocess.run(['strip', '--strip-all', '-o', binfile + '_stripped', binfile], cwd=directory)
  print('Stripping', '{:<24}'.format(binfile), '--> Return Code:', strip.returncode)

### File Size Analysis

The following code prints the file size of the different ''Hello World'' binaries.

In [ ]:
print('### File Size Analysis of Hellow World Binaries ###')
for [directory, binfile] in binaries:
    file_path = directory + binfile
    print('{:<84}'.format(file_path), '-->', '{:>9}'.format(os.path.getsize(file_path)), 'bytes')
    file_path_stripped = file_path + '_stripped'
    print('{:<84}'.format(file_path_stripped), '-->', '{:>9}'.format(os.path.getsize(file_path_stripped)), 'bytes')

### Section Analysis

This analysis shows interesting differences of statically and dynamically linked binaries in the sections ``.text``, ``.rodata``, ``.data`` and ``.bss``.

In [ ]:
print('### Dynamically Linked C Application ###')
size = subprocess.run(['size', '-A', 'hello_world_dyn_sizeopt'], cwd='./hello_world_c/', capture_output=True, text=True)
print('\n'.join(re.findall('section.*|\.text\s+.*|\.rodata\s+.*|\.data\s+.*|\.bss\s+.*', size.stdout)))
print()
print('### Statically Linked C Application ###')
size = subprocess.run(['size', '-A', 'hello_world_stat_sizeopt'], cwd='./hello_world_c/', capture_output=True, text=True)
print('\n'.join(re.findall('section.*|\.text\s+.*|\.rodata\s+.*|\.data\s+.*|\.bss\s+.*', size.stdout)))

Further, one can see the effects of stripping a Rust application.

It is clearly visible that all sections prefixed with ``.debug_`` were removed.

In [ ]:
print('### Statically Linked Rust Application (Before Stripping) ###')
size = subprocess.run(['size', '-A', 'hello_world'], cwd='./hello_world_rust/target/x86_64-unknown-linux-musl/release/', capture_output=True, text=True)
print(size.stdout)

print('### Statically Linked Rust Application (After Stripping) ###')
size = subprocess.run(['size', '-A', 'hello_world_stripped'], cwd='./hello_world_rust/target/x86_64-unknown-linux-musl/release/', capture_output=True, text=True)
print(size.stdout)

## Analysis of Prime Checking Applications in Rust

Checking numbers for primality is a common algorithm implementation case that leaves a lot of space for optimizations.

This exercise uses different versions of a command line tool for prime checking.

In [ ]:
prime_check_tools = [
    'prime_check_lookup_table',
    'prime_check_dyngen_table',
    'prime_check_no_table',
]

Let's compile all versions.

In [ ]:
print('### Compiling Prime Check Tools ###')
for tool in prime_check_tools:
  cargo = subprocess.run(['cargo', 'build', '--release'], cwd='./' + tool + '/')
  print('Compiling', '{:<24}'.format(tool), '--> Return Code:', cargo.returncode)

Let's also call ``strip`` on all binaries.

In [ ]:
print('### Stripping Binaries of Prime Check Tools ###')
for tool in prime_check_tools:
  strip = subprocess.run(['strip', '--strip-all', '-o', tool + '_stripped', tool], cwd='./' + tool + '/target/release/')
  print('Stripping', '{:<24}'.format(tool), '--> Return Code:', strip.returncode)

### File Size Analysis

Let's look at the file size of these binaries.

In [ ]:
print('### File Size Analysis of Prime Check Tools ###')
for tool in prime_check_tools:
    file_path = './' + tool + '/target/release/' + tool
    print('{:<84}'.format(file_path), '-->', '{:>9}'.format(os.path.getsize(file_path)), 'bytes')
    file_path = file_path + '_stripped'
    print('{:<84}'.format(file_path), '-->', '{:>9}'.format(os.path.getsize(file_path)), 'bytes')

### Section Analysis

Let's look a little bit closer, which section of ``prime_check_lookup_table`` shows the largest difference compared to the other two binaries.

In [ ]:
print('### Section Analysis of Prime Check Tools ###')
for tool in prime_check_tools:
    print('--- ' + tool + ' ---')
    size = subprocess.run(['size', '-A', tool], cwd='./' + tool + '/target/release/', capture_output=True, text=True)
    print('\n'.join(re.findall('section.*|\.text\s+.*|\.rodata\s+.*|\.data\s+.*|\.bss\s+.*', size.stdout)))